# 5.2. Tratamiento de series temporales II.

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib as plt
#%matplotlib inline

## Time Series con índices duplicados

- Podemos tener fechas duplicadas en el índice:

In [ ]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

In [ ]:
dup_ts.index.is_unique

In [ ]:
dup_ts['1/3/2000']  # not duplicated

In [ ]:
dup_ts['1/2/2000']  # duplicated

- Para eliminarlos podemos agrupar por nivel 0 y realizar una operación que los unifique.

In [ ]:
grouped = dup_ts.groupby(level=0)
grouped.mean()

In [ ]:
grouped.first()

In [ ]:
grouped.count()

### Date Offsets

In [ ]:
from pandas.tseries.offsets import Hour, Minute

In [ ]:
rango = pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')
rango

In [ ]:
rango + Hour(2) + Minute(30)

#### Desplazamiento

- Existen dos métodos principales: ``shift()`` and ``tshift()``
 * ``shift()`` desplaza los datos.
 * ``tshift()`` desplaza el índice.
- En ambos casos, se especifica por la frecuencia.

In [ ]:
dates = pd.date_range('2015-07-25', periods=15, freq='B')
data = pd.DataFrame({'close':[10,12,14,15,15,19, 20,17, 15, 14, 12,13,13,14,10]}, index=dates)
data

In [ ]:
data.plot()

In [ ]:
data.shift(5)

In [ ]:
data.tshift(5)

## Remuestreo
- La indexación de Pandas permite operaciones de remuestreo con relativa facilidad. 
- Es común necesitar obtener los datos a mayor o menor frecuencia de la disponible.
- Para ello se puede usar ``resample()`` o ``asfreq()`` (más sencillo).
- La diferencia principal es que mientras la primera es una agregación de datos, la segunda es una selección de los mismos. 

In [ ]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')
ts = pd.DataFrame(np.random.randn(len(rng)), index=rng)
ts

In [ ]:
ts.resample('M').mean()

In [ ]:
ts.resample('15D').first()

In [ ]:
ts.resample('BM').mean()

- Mientras que ``resample()`` devuelve la media del periodo
- ``asfreq()`` devuelve el último valor (omitiendo el mes en caso de no existir).
- Una ventaja adicional de ``asfreq()`` es la capacidad de imputar valores.

In [ ]:
ts.asfreq('5D')

### Downsampling

In [ ]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')
ts = pd.Series(np.arange(12), index=rng)
ts

In [ ]:
ts.resample('5min').sum()

In [ ]:
ts.resample('5min', closed='right').sum()

In [ ]:
ts.resample('5min', closed='right', label='right').sum()

### Resampling a Open-High-Low-Close (OHLC)

In [ ]:
ts

In [ ]:
ts.resample('5min').ohlc()

### Upsampling e Interpolation

In [ ]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', 
                                         periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

In [ ]:
df_daily = frame.resample('D').asfreq()
df_daily

In [ ]:
frame.resample('D').ffill()

In [ ]:
frame.resample('D').ffill(limit=2)

## Ventanas móviles

- Se utiliza la función ``rolling()``, que funciona de forma similar a ``groupby``.
- Con distintas operaciones de agregación disponibles.

In [ ]:
close_px_all = pd.read_csv('test_data/stock_px_2.csv',
                           parse_dates=True,
                           index_col=0)
close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
close_px.head()

In [ ]:
close_px.plot()

In [ ]:
close_px = close_px.resample('B').ffill()
close_px

In [ ]:
close_px.AAPL.plot()
close_px.AAPL.rolling(250).mean().plot()
close_px.AAPL.rolling(30).mean().plot()

In [ ]:
close_px['rmean'] = close_px.AAPL.rolling(250, min_periods=10).std()

In [ ]:
close_px

In [ ]:
appl_std250 = close_px.AAPL.rolling(250, min_periods=10).std()
appl_std250[5:12]

In [ ]:
appl_std250.plot()

In [ ]:
std_ret = close_px.AAPL.pct_change().rolling(30).std()
std_ret.plot()

#### Ejemplo Bandas Bollinger

In [ ]:
apple_price = close_px.AAPL['2010']

mv_avg = apple_price.rolling(30).mean()
std_ret = apple_price.rolling(30).std()

upper_band = mv_avg + 2*std_ret
lower_band = mv_avg - 2*std_ret

apple_price.plot()
mv_avg.plot()
upper_band.plot()
lower_band.plot()

### Ventanas expansivas
- Ventanas que se expanden en su tamaño cada elemento.

In [ ]:
expanding_max = close_px.AAPL.expanding().max()

In [ ]:
close_px.AAPL.plot()
expanding_max.plot()

In [ ]:
((close_px.AAPL / expanding_max)-1).plot()

In [ ]:
expanding_max.plot()
close_px.AAPL.plot()

In [ ]:
close_px.rolling(60).mean().plot(logy=True)

In [ ]:
close_px.rolling('20D').mean()

In [ ]:
close_px.AAPL.resample('Y').ohlc()

### Exponentially Weighted

In [ ]:
aapl_px = close_px.AAPL['2006']
ma60 = aapl_px.rolling(30, min_periods=20).mean()
ewma60 = aapl_px.ewm(span=30).mean()

In [ ]:
aapl_px.plot()
ma60.plot(style='b--', label='Simple MA')
ewma60.plot(style='k-', label='EW MA')

### Ejemplo: Correlación Rolling 

In [ ]:
spx_close = close_px_all.loc[:, 'SPX']
appl_close = close_px_all.loc[:, 'AAPL']

# natural returns
# spx_returns = spx_close.pct_change()
# appl_returns = appl_close.pct_change()

# Log returns
spx_returns = np.log(spx_close).diff()
appl_returns = np.log(appl_close).diff()

In [ ]:
corr = appl_returns.rolling(250, min_periods=10).corr(spx_returns)
corr.plot()

___
# Ejercicios

**5.2.1.** Carga los csvs de datos ibex_div, ibex, NTGY, REE, SAN, pon la fecha como índice.

**5.2.2.**  Calcula el retorno anualizado del Ibex con dividendos y del Ibex.

**5.2.3.**  Calcula la serie de retornos anuales  del Ibex con dividendos y del Ibex.

**5.2.4.**  Realiza un gráfico de barras comparándolos.

**5.2.5.**  Compara los retornos anuales del Ibex con los de SAN.

**5.2.6.**  Calcula la correlación del SAN, REE y NTGY con el IBEX con dividendos.

**5.2.7.** Ahora calcula la correlación rolada de 100 días para los activos del ejercico anterior.

**5.2.8.** Calcula la media movil de 30 y 200 dias de REE y realiza una figura junto con la serie de precios originales

**5.2.9.**  Usando el precio de cierre del Ibex, calcula las velas mensuales y anuales.

**5.2.10.** Píntalas utilizando un gráfico de barras.

**5.2.11.**  Calcula la beta de los 3 activos NTGY, REE y SAN, Recuerda:

$\beta = \frac{cov(R_m, R_s)}{var(R_m)}$

Donde $R_m$ y $R_s$ son la serie de retornos del índice y de la acción.

**5.2.12.** Ahora calcula la correlación rolada de 100 días para los activos del ejercico anterior.